In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [32]:
# Retrieve the parent divs for all articles
#results = soup.find_all('div', class_="image_and_description_container")
#
#for result in results:
#    
#    # scrape the article subheader
#    news_synopsis = result.find('a').find('div', class_='rollover_description_inner').text
#    
#    # scrape the datetime
#    #news_datetime = result.find('div', class_='list_text')
#    
#    # scrape the article header 
#    news_title = result.find('div', class_='content_title')
#    
#    # print article data
#    print('-----------------')
#    #print(news_title)
#    print(news_synopsis)
#    #print(news_datetime)

In [33]:
# The article header
news_title = soup.find('div', class_="content_title").text
print(news_title)



NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network




In [34]:
news_synopsis = soup.find('div', class_='rollover_description_inner')
print(news_synopsis.text)


Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.



In [24]:
for row in soup.find_all('div',attrs={"class" : "article_teaser_body"}):
    print(row)